In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd


## NASA Mars News

In [57]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'lxml')

In [58]:
results = soup.find_all('section', class_='grid_layout')
results


[]

## JPL Mars Space Images - Featured Image

In [59]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/dvadakan/.wdm/drivers/chromedriver/mac64/87.0.4280.20/chromedriver] found in cache


In [60]:
# USe Splinter to navigate to the JPL Featured Space Image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [61]:
# Find the link to featured image
browser.links.find_by_partial_text('FULL IMAGE').click()


In [62]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# find the featured image
featured_image_url = "https://www.jpl.nasa.gov" + soup.find('img', class_='fancybox-image')['src']
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19041_ip.jpg


In [63]:
browser.quit()

## Mars Facts

In [66]:
spacefacts_url = 'https://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
tables = pd.read_html(spacefacts_url)

# Find the Mars facts table and convert to pandas dataframe
mars_df = tables[0]
mars_df.columns = ["Attributes", "Values"]
mars_df.set_index("Attributes", inplace=True)
mars_df

,Values
Attributes,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [68]:
# Convert the table to html
mars_df.to_html("mars_facts.html")

## Mars Hemispheres

In [63]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/dvadakan/.wdm/drivers/chromedriver/mac64/87.0.4280.20/chromedriver] found in cache


In [64]:
# USe Splinter to navigate to the JPL Featured Space Image
jpl_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(jpl_url)
jpl_html = browser.html
soup = BeautifulSoup(jpl_html, 'html.parser')

In [65]:
# create empty list for images and titles
hemisphere_image_urls = []

# Find the 4 hemisphere images and titles
for i in [1,3,5,7]:
    
    # Find all the links to the hemispheres
    results = browser.links.find_by_partial_href('/search/map/Mars/Viking')
    
    # Click on the image link to find full size image
    results[i].click()

    # HTML object
    html = browser.html

    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Retrieve list element that contain image
    li_element = soup.find_all('li')
    image_url = li_element[0].find('a')['href']
    print(image_url)
    
    # append image link to list
    #image_urls.append(image_url)
    
    # Retrieve h2 element that contains title
    title = soup.find('h2', class_='title').text
    print(title)
    
    #append title to list
    #titles.append(title)
    hemisphere_image_urls_dict = {}
    hemisphere_image_urls_dict["title"] = title
    hemisphere_image_urls_dict["image_url"] = image_url
    hemisphere_image_urls.append(hemisphere_image_urls_dict)

    
    # go back to the main page
    browser.back()

hemisphere_image_urls

browser.quit()

https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Cerberus Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced


[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]